In [1]:
! pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 6.3 MB 8.4 MB/s eta 0:00:01     |▏                               | 40 kB 15.3 MB/s eta 0:00:01
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23459 sha256=44979a0d3edf068fbb5bea17b2d40160de8594dd45d89076e93267191ccd98f3
  Stored in directory: /home/stra10/.cache/pip/wheels/b5/12/81/97d5fe1c20677971f3ad0d93e7b8b38bb7ac85496c17a13343
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=6582 sha256=00f3373519d7e9270c5352ab82f225fc98275029af93408c22d30016bf85d89d
  Stored in directory: /home/stra10/.cache/pip/wheels/5e/8a/c4/59c699498647c7c94b14c87a904ca7540646107b3d94b7c320
Successfully built yfinance multitasking
  Attempting uninstall: lxml
    Found existing installation: lxml 4.4.1
    Uninstalling lxml-4.4.1:
      Successfully uninstalled lxml-4.4.1


In [3]:
! pip install pandas_datareader

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 107 kB 4.3 MB/s eta 0:00:01


# Importing necessary libraries

In [4]:
from bs4 import BeautifulSoup as BS
import requests
import yfinance as yf
import datetime
import pandas as pd
import pandas_datareader.data as web

import urllib.request as urllib2 
from urllib.request import URLError

from tqdm import tqdm

import os 
import pickle
import numpy as np

import copy
import math


# Getting the list of tickers

In [5]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BS(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2021,4,28)
end = datetime.datetime(2021,5,28)


# Getting market cap data for each ticker

In [6]:
dates_keys = ['2021-04-28', '2021-04-29', '2021-04-30',  '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06']
dates_keys = dates_keys + ['2021-05-07', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-13', '2021-05-14', '2021-05-17', '2021-05-18', '2021-05-19']
dates_keys = dates_keys + ['2021-05-20', '2021-05-21', '2021-05-24', '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28']


23

# Try to load data 

In [7]:
if 'market_cap.pkl' in os.listdir('.'):
  with open('market_cap.pkl', 'rb') as file:
    market_cap_dict = pickle.load(file)

# If there is no data to load -> download it yourself

In [8]:
if 'market_cap.pkl' not in os.listdir('.'):

  market_cap_dict = {}

  for ticker in tqdm(tickers):
    soup = BS(urllib2.urlopen('https://ycharts.com/companies/' + ticker + '/market_cap'), features="html.parser")
    leftCol = soup.find('div', {'class':'dataColLeft'})
    dates = leftCol.find_all('td', {'class':'col1'})
    market_caps = leftCol.find_all('td', {'class':'col2'})

    dates = list(map(lambda x: x.string, dates))
    market_caps = list(map(lambda x: x.string.strip(), market_caps))

    zipped = list(zip(dates, market_caps))

    # preprocess the data
    dates = []
    for zipp in zipped:
      month_day, year = zipp[0].split(', ')
      month, day = month_day.split(' ')
      dates.append((year + ('-05-' if month == 'May' else '-04-') + ('0' if int(day) < 10 else '') + day))

    prices = []
    for zipp in zipped:
      value = int(zipp[1].replace('.', '').replace('T', '000000000').replace('B', '000000'))
      prices.append(value)

    marketCapKnown = dict(list(zip(dates, prices)))

    marketCap = dict([(dk, 0) for dk in dates_keys])
    marketCap.update(marketCapKnown)

    market_cap_dict[ticker] = marketCap


100%|██████████| 505/505 [07:26<00:00,  1.13it/s]


# Save data to reuse it later

In [9]:
with open('market_cap.pkl', 'wb') as file:
  pickle.dump(market_cap_dict, file)

# Download yfinance data

In [10]:
data = yf.download(tickers, start=start, end=end)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [11]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Get Adj Close data

In [45]:
adj_close_data = data['Adj Close']

# Calculating change in percentages

In [13]:
ptc_changes = adj_close_data.pct_change()
ptc_changes = ptc_changes.dropna(how='all')
ptc_changes = ptc_changes.dropna(axis=1,how='all')
ptc_changes

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,CPB,CPRT,CRL,CRM,CSCO,CSX,CTAS,CTLT,CTSH,CTVA,CTXS,CVS,CVX,CZR,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,ENPH,EOG,EQIX,EQR,ES,ESS,ETN,ETR,ETSY,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLT,FMC,FOX,FOXA,FRC,FRT,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GNRC,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,IP,IPG,IPGP,IQV,IR,IRM,ISRG,IT,ITW,IVZ,J,JBHT,JCI,JKHY,JNJ,JNPR,JPM,K,KEY,KEYS,KHC,KIM,KLAC,KMB,KMI,KMX,KO,KR,KSU,L,LB,LDOS,LEG,LEN,LH,LHX,LIN,LKQ,LLY,LMT,LNC,LNT,LOW,LRCX,LUMN,LUV,LVS,LW,LYB,LYV,MA,MAA,MAR,MAS,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MKTX,MLM,MMC,MMM,MNST,MO,MOS,MPC,MPWR,MRK,MRO,MS,MSCI,MSFT,MSI,MTB,MTD,MU,MXIM,NCLH,NDAQ,NEE,NEM,NFLX,NI,NKE,NLOK,NLSN,NOC,NOV,NOW,NRG,NSC,NTAP,NTRS,NUE,NVDA,NVR,NWL,NWS,NWSA,NXPI,O,ODFL,OKE,OMC,ORCL,ORLY,OTIS,OXY,PAYC,PAYX,PBCT,PCAR,PEAK,PEG,PENN,PEP,PFE,PFG,PG,PGR,PH,PHM,PKG,PKI,PLD,PM,PNC,PNR,PNW,POOL,PPG,PPL,PRGO,PRU,PSA,PSX,PTC,PVH,PWR,PXD,PYPL,QCOM,QRVO,RCL,RE,REG,REGN,RF,RHI,RJF,RL,RMD,ROK,ROL,ROP,ROST,RSG,RTX,SBAC,SBUX,SCHW,SEE,SHW,SIVB,SJM,SLB,SNA,SNPS,SO,SPG,SPGI,SRE,STE,STT,STX,STZ,SWK,SWKS,SYF,SYK,SYY,T,TAP,TDG,TDY,TEL,TER,TFC,TFX,TGT,TJX,TMO,TMUS,TPR,TRMB,TROW,TRV,TSCO,TSLA,TSN,TT,TTWO,TWTR,TXN,TXT,TYL,UA,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,V,VFC,VIAC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-28,-0.012309,-0.002757,0.001774,-0.006027,0.004397,0.004461,0.030943,-0.002054,0.000069,-0.003557,-0.021874,0.020705,-0.047322,0.004566,0.002397,-0.005390,0.006115,-0.000187,-0.004748,-0.011193,0.010607,-0.023652,-0.002062,-0.013510,-0.006225,-0.003894,-0.008354,-0.001975,-0.007969,0.001714,-0.013966,-0.000222,-0.072198,0.000351,-0.002139,0.012018,0.005454,0.005060,-0.004193,0.019633,-0.023194,0.076721,-0.000690,-0.010841,0.001431,0.002809,-0.011628,-0.003821,-0.020275,-0.015332,0.035106,-0.004900,0.014498,0.002892,-0.028911,0.003761,0.002651,-0.009801,-0.002711,0.010396,0.000785,0.000493,0.012976,0.016618,0.016566,-0.005155,-0.016685,0.001061,-0.001365,0.043150,0.006985,-0.005180,-0.001096,-0.001889,0.005355,-0.010454,0.008509,-0.015246,-0.012214,-0.010182,0.002436,-0.000360,0.003917,-0.007957,-0.002172,-0.007151,0.011635,0.010282,-0.004365,-0.030645,-0.001224,-0.013177,-0.015402,0.002673,-0.006656,0.022582,-0.003683,-0.018361,0.004963,-0.009502,-0.004264,-0.029685,0.005460,0.038935,0.010721,-0.000884,0.033918,-0.001675,-0.009371,-0.005566,0.001286,0.011400,-0.005061,-0.004121,-0.014137,0.004347,0.003002,0.000610,-0.013040,0.003861,0.024538,0.008156,-0.008302,0.002158,-0.000389,-0.006695,0.034892,-0.001507,-0.004686,-0.007656,0.001434,-0.006770,-0.038964,-0.036479,-0.020490,-0.005138,-0.008434,0.001082,0.010535,0.007281,0.004215,0.008482,0.002422,-0.000406,0.008455,0.08

In [32]:
number_of_companies = len(ptc_changes.columns)
changes_of_prices = ptc_changes.sum(axis=1)

index_change_ptc = changes_of_prices/number_of_companies+1

In [33]:
index_change_ptc_df=pd.DataFrame(index_change_ptc)

# Projecting and comparing the projection to the real values

In [34]:
sp_curr=4183.18  # s&p index value on the starting date
sp_list=[sp_curr]
for ind in index_change_ptc:
  sp_curr=sp_curr*ind
  sp_list.append(sp_curr)

In [35]:
SP500 = web.DataReader(['sp500'], 'fred', start, end)

In [37]:
real_SP500 = SP500.dropna(axis=0,how='all')
real_SP500 = real_SP500


In [38]:
sp_projection_df = pd.DataFrame(sp_list, index=real_SP500.index)

In [40]:
comparison_ew=pd.concat([real_SP500,sp_projection_df],axis=1)
comparison_ew


,sp500,0
DATE,,
2021-04-28,4183.18,4183.180000
2021-04-29,4211.47,4183.419451
2021-04-30,4181.17,4213.015415
2021-05-03,4192.66,4179.560152
2021-05-04,4164.66,4204.827224
2021-05-05,4167.59,4197.850257
2021-05-06,4201.62,4209.865505
2021-05-07,4232.60,4230.685729
2021-05-10,4188.43,4275.867953


## Equal weighed influence is not the best approach.

#### Because of that, we are going to calculate the weight for each ticker as the ratio of it's number of shares and the total number of shares of all tickers.

\begin{align}
W_{ticker} & = \frac{number\_of\_shares_{ticker}}{total\_number\_of\_shares}
\end{align}



# Involving market capitalization

In [51]:
market_cap_df = pd.DataFrame.from_dict(market_cap_dict)

# sort rows by date
market_cap_df = market_cap_df.sort_index()
# sort columns by name (i.e date)
market_cap_df = market_cap_df.reindex(sorted(market_cap_df.columns), axis=1)




In [56]:
# removing non-interesting dates 
# market_cap_df = market_cap_df.iloc[3:-1]

# Dividing the market capitalization with stock price to get the number of shares on the given date


\begin{align}
number\_of\_shares_{ticker} & = \frac{market\_cap_{ticker}}{share\_price_{ticker}}
\end{align}



In [59]:
number_of_shares_per_ticker_df = market_cap_df.iloc[1:-1].div(adj_close_data)

# Calculating the ratio of the number-of-shares per ticker and total-number-of-shares

In [71]:
number_of_shares_per_ticker_list = number_of_shares_per_ticker_df.values.tolist()
number_of_shares_per_ticker_list = list(map(lambda list_: [0 if math.isnan(i) else i for i in list_], number_of_shares_per_ticker_list))
number_of_shares_per_ticker_sums = list(map(sum, number_of_shares_per_ticker_list))

number_of_shares_ptc = copy.copy(number_of_shares_per_ticker_df)

for i in range(len(number_of_shares_per_ticker_df)):
  number_of_shares_ptc.iloc[i] = number_of_shares_per_ticker_df.iloc[i] / number_of_shares_per_ticker_sums[i]


In [74]:
number_of_shares_ptc

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,CPB,CPRT,CRL,CRM,CSCO,CSX,CTAS,CTLT,CTSH,CTVA,CTXS,CVS,CVX,CZR,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,ENPH,EOG,EQIX,EQR,ES,ESS,ETN,ETR,ETSY,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLT,FMC,FOX,FOXA,FRC,FRT,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GNRC,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,IP,IPG,IPGP,IQV,IR,IRM,ISRG,IT,ITW,IVZ,J,JBHT,JCI,JKHY,JNJ,JNPR,JPM,K,KEY,KEYS,KHC,KIM,KLAC,KMB,KMI,KMX,KO,KR,KSU,L,LB,LDOS,LEG,LEN,LH,LHX,LIN,LKQ,LLY,LMT,LNC,LNT,LOW,LRCX,LUMN,LUV,LVS,LW,LYB,LYV,MA,MAA,MAR,MAS,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MKTX,MLM,MMC,MMM,MNST,MO,MOS,MPC,MPWR,MRK,MRO,MS,MSCI,MSFT,MSI,MTB,MTD,MU,MXIM,NCLH,NDAQ,NEE,NEM,NFLX,NI,NKE,NLOK,NLSN,NOC,NOV,NOW,NRG,NSC,NTAP,NTRS,NUE,NVDA,NVR,NWL,NWS,NWSA,NXPI,O,ODFL,OKE,OMC,ORCL,ORLY,OTIS,OXY,PAYC,PAYX,PBCT,PCAR,PEAK,PEG,PENN,PEP,PFE,PFG,PG,PGR,PH,PHM,PKG,PKI,PLD,PM,PNC,PNR,PNW,POOL,PPG,PPL,PRGO,PRU,PSA,PSX,PTC,PVH,PWR,PXD,PYPL,QCOM,QRVO,RCL,RE,REG,REGN,RF,RHI,RJF,RL,RMD,ROK,ROL,ROP,ROST,RSG,RTX,SBAC,SBUX,SCHW,SEE,SHW,SIVB,SJM,SLB,SNA,SNPS,SO,SPG,SPGI,SRE,STE,STT,STX,STZ,SWK,SWKS,SYF,SYK,SYY,T,TAP,TDG,TDY,TEL,TER,TFC,TFX,TGT,TJX,TMO,TMUS,TPR,TRMB,TROW,TRV,TSCO,TSLA,TSN,TT,TTWO,TWTR,TXN,TXT,TYL,UA,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,V,VFC,VIAC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VTRS,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
2021-04-27,0.000499,0.001050,0.000107,0.273739,0.002891,0.000337,0.000074,0.002909,0.001041,0.000783,0.000607,0.000920,0.000697,0.000360,0.000418,0.000825,0.001097,0.001119,0.001407,0.000997,0.000321,0.000267,0.000191,0.000130,0.002037,0.000490,0.000147,0.000362,0.001505,0.002544,0.001989,0.000378,0.000947,0.000191,0.000728,0.008255,0.000125,0.000143,0.000401,0.000371,0.000266,0.006189,0.000362,0.000978,0.000443,0.000242,0.000215,0.001272,0.000229,0.000669,0.000136,0.000298,0.001315,0.000036,0.000958,0.014064,0.000823,0.000410,0.000476,0.000826,NaN,0.000247,0.000049,0.001433,0.000067,0.001718,0.000250,0.000538,0.003656,0.000190,NaN,0.002326,0.000393,0.000256,0.003408,0.000792,0.000476,0.001427,0.000897,0.000736,0.000175,0.000550,0.000707,0.001949,0.000456,0.000230,0.000184,0.000493,0.000353,0.000704,0.000403,0.000218,0.000309,0.000564,0.000264,0.001385,0.000204,0.000229,0.007522,0.000588,0.000046,0.000241,0.000477,0.000954,0.000909,0.000750,0.006581,0.000080,0.002227,0.000726,0.000496,0.000387,0.000082,0.001510,0.006912,0.001244,0.000172,0.000279,0.000868,0.001213,0.000203,0.002156,0.003195,0.000342,0.001320,0.001047,0.000874,0.000513,0.000503,0.000392,0.000199,0.000591,0.001168,0.002916,0.000827,0.000827,0.000863,0.000461,0.000383,0.000236,0.001236,0.000064,0.000616,0.000214,0.000317,0.001272,0.000174,0.001109,0.004169,0.000158,0.000468,0.001119,0.000468,0.000566,0.000200,0.000621,0.000595,0.000223,0.000987,0.000222,0.000955,0.000147,0.000612,0.000566,0.000106,0.000656,0.000331,0.000208,0.000379,0.001018,0.001614,0.000278,0.000241,0.000219,0.006536,0.000298,0.000941,0.004643,0.000228,0.002398,0.000434,0.000900,0.000098,0.001015,0.001092,0.001165,0.000136,0.000211,0.000954,0.000954,0.000289,0.001272,0.000267,0.000555,0.000463,0.014373,0.002053,0.000999,0.000169,0.001401,0.002375,0.000103,0.010972,0.010970,0.000237,0.000483,0.000614,0.000315,0.000558,0.000086,0.00145

# Do the dot product of the price change and calculated weights


#### First, make sure that the lists of tickers match completely

In [75]:
columns_for_removal = []
for share_ticker in number_of_shares_ptc.columns:
  if share_ticker not in ptc_changes.columns:
    columns_for_removal.append(share_ticker)

number_of_shares_ptc = number_of_shares_ptc.drop(columns_for_removal, axis=1)


#### Finally, calculate the dot product

In [76]:
number_of_shares_ptc_list = number_of_shares_ptc.values.tolist()
ptc_change_list = ptc_changes.values.tolist()

lists = list(zip(number_of_shares_ptc_list, ptc_change_list))
weighted_change = list(map(lambda x: 1+sum(i[0] * i[1] for i in zip(x[0], x[1])), lists))
weighted_change


[0.9984008279182489,
 1.0041344517046602,
 0.990081119570935,
 1.0067395247853144,
 0.9875675288240846,
 1.0040536328454284,
 1.0095047623079425,
 1.0087031027663522,
 0.9878356610511635,
 0.9901250148066542,
 0.9787091555902088,
 1.01331113111832,
 1.019091192560454,
 0.9979592971944344,
 0.9895163043766416,
 0.9953515342541842,
 1.0104532572614764,
 0.9970286003927024,
 1.0097628538546952,
 0.9955602146084279,
 1.0038883269723278,
 0.999964397745238]

# Comparing outputs

In [77]:
sp_curr=4183.18  # s&p index value on the starting date
sp_list=[sp_curr]
for ind in weighted_change:
  sp_curr=sp_curr*ind
  sp_list.append(sp_curr)

In [78]:
weighted_sp_projection_df = pd.DataFrame(sp_list, index=real_SP500.index)

In [83]:
comparison=pd.concat([ weighted_sp_projection_df, comparison_ew],axis=1)
comparison.set_axis(['Weighted Projection', 'Real S&P500', 'Equal Weighted Projection'], axis=1, inplace=True)
comparison

,Weighted Projection,Real S&P500,Equal Weighted Projection
DATE,,,
2021-04-28,4183.180000,4183.18,4183.180000
2021-04-29,4176.490375,4211.47,4183.419451
2021-04-30,4193.757873,4181.17,4213.015415
2021-05-03,4152.160490,4192.66,4179.560152
2021-05-04,4180.144079,4164.66,4204.827224
2021-05-05,4128.174558,4167.59,4197.850257
2021-05-06,4144.908662,4201.62,4209.865505
2021-05-07,4184.305034,4232.60,4230.685729
2021-05-10,4220.721470,4188.43,4275.867953


## Calculating the squared error

In [80]:
sum_sq_w = 0
sum_sq_e = 0

for triple in comparison.values.tolist():
  sum_sq_w += (triple[0] - triple[1])**2
  sum_sq_e += (triple[2] - triple[1])**2

length = len(comparison.values.tolist())
sum_sq_w /= length
sum_sq_e /= length  

(42.23527108552451, 58.586144661488234)

In [85]:

np.sqrt(sum_sq_w), np.sqrt(sum_sq_e)
print('Equal-weigth-projection squared error: ', np.sqrt(sum_sq_e))
print('Weigthed-projection squared error: ', np.sqrt(sum_sq_w))

Equal-weigth-projection squared error:  58.586144661488234
Weigthed-projection squared error:  42.23527108552451
